In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway, shapiro, levene, kruskal, sem
import scipy.stats as st 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import numpy as np
import string
from matplotlib.backends.backend_pdf import PdfPages


In [63]:
# Manditory column order ['date', 'country', 'Subject', 'events']
dfc = pd.read_csv('search_data_pest.csv')
sub = dfc.columns[2]
print(dfc.columns)

Index(['date', 'country', 'crop', 'events'], dtype='object')


In [64]:
# Format date column
dfc['date'] = pd.to_datetime(dfc['date'], format='%d-%b-%y', dayfirst=True)
print(dfc.head())

print(dfp.head())

        date country                           crop  events
0 2023-09-30   India                     Leaf miner       1
1 2023-09-30    Peru              Planococcus citri       1
2 2023-09-30   India                Citrus mealybug       1
3 2023-09-30   India  Brinjal fruit and shoot borer       1
4 2023-09-30  Canada                         Thrips       1
        date country                           pest  events
0 2023-09-30   India                     Leaf miner       1
1 2023-09-30    Peru              Planococcus citri       1
2 2023-09-30   India                Citrus mealybug       1
3 2023-09-30   India  Brinjal fruit and shoot borer       1
4 2023-09-30  Canada                         Thrips       1


<font size="5"> Compact letter display algorithm</font>

In [65]:
def cld_al(df, alpha=0.1):

    df["p-adj"] = df["p-adj"].astype(float)

    # Creating a list of the different treatment groups from Tukey's
    group1 = set(df.group1.tolist())  # Dropping duplicates by creating a set
    group2 = set(df.group2.tolist())  # Dropping duplicates by creating a set
    groupSet = group1 | group2  # Set operation that creates a union of 2 sets
    groups = sorted(list(groupSet))

    # Creating lists of letters that will be assigned to treatment groups
    letters = list(string.ascii_lowercase)[:len(groups)]
    cldgroups = letters

    # the following algoritm is a simplification of the classical cld,

    cld = pd.DataFrame(list(zip(groups, letters, cldgroups)))
    cld[3]=""
    
    for row in df.itertuples():
        if df["p-adj"][row[0]] > (alpha):
            cld.iat[groups.index(df["group1"][row[0]]), 2] += cld.iat[groups.index(df["group2"][row[0]]), 1]
            cld.iat[groups.index(df["group2"][row[0]]), 2] += cld.iat[groups.index(df["group1"][row[0]]), 1]
            
        if df["p-adj"][row[0]] < (alpha):
                cld.iat[groups.index(df["group1"][row[0]]), 3] +=  cld.iat[groups.index(df["group2"][row[0]]), 1]
                cld.iat[groups.index(df["group2"][row[0]]), 3] +=  cld.iat[groups.index(df["group1"][row[0]]), 1]

    cld[2] = cld[2].apply(lambda x: "".join(sorted(x)))
    cld[3] = cld[3].apply(lambda x: "".join(sorted(x)))
    cld.rename(columns={0: "groups"}, inplace=True)

    # this part will reassign the final name to the group
    # for sure there are more elegant ways of doing this
    cld = cld.sort_values(cld.columns[2], key=lambda x: x.str.len())
    cld["labels"] = ""
    letters = list(string.ascii_lowercase)
    unique = []
    for item in cld[2]:

        for fitem in cld["labels"].unique():
            for c in range(0, len(fitem)):
                if not set(unique).issuperset(set(fitem[c])):
                    unique.append(fitem[c])
        g = len(unique)

        for kitem in cld[1]:
            if kitem in item:
                if cld["labels"].loc[cld[1] == kitem].iloc[0] == "":
                    cld["labels"].loc[cld[1] == kitem] += letters[g]

                #Checking if there are forbidden pairing (proposition of solution to the imperfect script)                
                if kitem in ' '.join(cld[3][cld["labels"]==letters[g]]): 
                    g=len(unique)+1
               
                # Checking if columns 1 & 2 of cld share at least 1 letter
                if len(set(cld["labels"].loc[cld[1] == kitem].iloc[0]).intersection(cld.loc[cld[2] == item, "labels"].iloc[0])) <= 0:
                    if letters[g] not in list(cld["labels"].loc[cld[1] == kitem].iloc[0]):
                        cld["labels"].loc[cld[1] == kitem] += letters[g]
                    if letters[g] not in list(cld["labels"].loc[cld[2] == item].iloc[0]):
                        cld["labels"].loc[cld[2] == item] += letters[g]

    cld = cld.sort_values("labels")
    #print(cld)
    #print('\n')
    cld.drop(columns=[1, 2, 3], inplace=True)
    #print(cld)
    #print('\n')
    #print('\n')
    return(cld)


<font size="5"> ANOVA loop</font>

In [93]:
# Create df ANOVA for results  
test_results = pd.DataFrame(columns=['test', 'country', sub, 'F/H-statistic', 'p-value'])


country_index = 0


# Testing loop 
for country in dfc['country'].unique():
    filtered_country_df = dfc[dfc['country'] == country]
    country_index += 1
    print('country ', country_index, '/', len(dfc['country'].unique()), ' Started.')
    sub_index = 0
    
    for subject in filtered_country_df[sub].unique():
        filtered_df = filtered_country_df[filtered_country_df[sub] == subject]
        sub_index +=1
        print('Subject ', sub_index,'/', len(filtered_country_df[sub].unique()), 'in country ', country_index,'/',len(dfc['country'].unique()), ' Started.')

        # Test data assumptions/requirements
        if len(filtered_df['date'].dt.month.unique()) >= 3:
            print('groups for ', country, 'and ', subject, 'are sufficient')
            # Specify the date range
            start_date = '2021-10-01'
            end_date = '2023-09-30'
            date_range = pd.date_range(start=start_date, end=end_date, freq='D')

            # Create a df with the date range
            date_range_df = pd.DataFrame({'date': date_range})
 
            # Merge existing data with the date range, filling missing values with zeros and specified crop name
            filtered_df = pd.merge(date_range_df, filtered_df, on='date', how='left').fillna({'events': 0, sub : subject, 'country': country})
            filtered_df['month'] = filtered_df['date'].dt.month
            filtered_df['year'] = filtered_df['date'].dt.year
            
            #Average data by month 
            filtered_df['events'] = pd.to_numeric(filtered_df['events'], errors='coerce')
            
            
            # Check normality
            _, p_normality = shapiro(filtered_df['events'])
            print(country , subject , p_normality)
            # Check homogeneity of variances
            _, p_homogeneity = levene(filtered_df['events'], filtered_df['month'])
            print(country , subject , p_homogeneity)

            # check p values and proceed as required
            if p_normality < 0.05 and p_homogeneity < 0.05:

                
                # The data assumptions are met so proceed with ANOVA test 
                print('ANOVA performed for ', country, 'and ', subject,)
                f_statistic, p_value = f_oneway(filtered_df['month'], filtered_df['events'])
                dict_res ={ 
                    'test': 'ANOVA',
                    'country': country,
                    sub : subject,
                    'F/H-statistic': float(f_statistic),
                    'p-value': float(p_value)
                }
                test_results = (
                    pd.DataFrame([dict_res]).copy() if test_results.empty 
                    else pd.concat([test_results, pd.DataFrame([dict_res])], ignore_index=True)
                )
            
            #The data assumptions are not met so proceed with the Kruskal-Wallis test 
            else:
                #Proceed with the Kruskal-Wallis test 
                print('kruskal performed for ', country, 'and ', subject,)
                h_statistic, p_value_kruskal = kruskal(filtered_df['month'], filtered_df['events'])
                #print(p_value_kruskal)
                dict_res ={ 
                    'test': 'Kruskal',
                    'country': country,
                    sub : subject,
                    'F/H-statistic': float(h_statistic),
                    'p-value': float(p_value_kruskal)
                }
             
                test_results = (
                    pd.DataFrame([dict_res]).copy() if test_results.empty 
                    else pd.concat([test_results, pd.DataFrame([dict_res])], ignore_index=True)
                )
                
        # Not enough samples to run the test
        else:
            print('not enough for ', country, 'and ', subject)
            dict_res ={ 
                    'test': 'Not enough data',
                    'country': country,
                    sub : subject,
                    'F/H-statistic': 0,
                    'p-value': 0
                }
            
            test_results = (
                    pd.DataFrame([dict_res]).copy() if test_results.empty 
                    else pd.concat([test_results, pd.DataFrame([dict_res])], ignore_index=True)
                )

test_results.to_csv(sub+' results.csv', index=False)
print('complete')

country  1 / 165  Started.
Subject  1 / 473 in country  1 / 165  Started.
groups for  India and  Leaf miner are sufficient
India Leaf miner 1.401298464324817e-45
India Leaf miner 1.3809306101583667e-250
ANOVA performed for  India and  Leaf miner
Subject  2 / 473 in country  1 / 165  Started.
groups for  India and  Citrus mealybug are sufficient
India Citrus mealybug 0.0
India Citrus mealybug 7.726776488875463e-269
ANOVA performed for  India and  Citrus mealybug
Subject  3 / 473 in country  1 / 165  Started.
groups for  India and  Brinjal fruit and shoot borer are sufficient
India Brinjal fruit and shoot borer 1.401298464324817e-44
India Brinjal fruit and shoot borer 1.1973953432825746e-204
ANOVA performed for  India and  Brinjal fruit and shoot borer
Subject  4 / 473 in country  1 / 165  Started.
groups for  India and  Fruit borer are sufficient
India Fruit borer 0.0
India Fruit borer 5.98960062889611e-242
ANOVA performed for  India and  Fruit borer
Subject  5 / 473 in country  1 / 165

<font size="5"> Tukey loop (ANOVA follow up)</font>

In [ ]:
#ANOVA follow up
anova_results = test_results[test_results['test'] == 'ANOVA']
tukey_results = pd.DataFrame(columns=['country', sub, 'group1', 'group2', 'meandiff', 'p-adj', 'lower', 'upper', 'reject'])
graph_data_df = pd.DataFrame(columns=['country', sub,'month', 'events_mean', 'events_std','confidence_interval', 'labels', 'groups'])
test_index = 0

for index, row in anova_results.iterrows():
    
    test_index += 1 
    print('test ', test_index, '/', len(anova_results), ' started')
    
    p_value = row['p-value']
    
    if p_value < 0.1:
        
        filtered_anova_df = dfc[(dfc['country'] == row['country']) & (dfc[sub] == row[sub])]
        temp_country = row['country']
        temp_crop = row[sub]
      
        # Specify the date range
        start_date = '2021-10-01'
        end_date = '2023-09-30'
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')

        # Create a df with the date range
        date_range_df = pd.DataFrame({'date': date_range})
 
        # Merge existing data with the date range, filling missing values with zeros and specified crop name
        filtered_anova_df = pd.merge(date_range_df, filtered_anova_df, on='date', how='left').fillna({'events': 0, sub : row[sub], 'country': row['country']})
        filtered_anova_df['month'] = filtered_anova_df['date'].dt.month
        filtered_anova_df['year'] = filtered_anova_df['date'].dt.year

        # Average data by month 
        filtered_anova_df['events'] = pd.to_numeric(filtered_anova_df['events'], errors='coerce')
        filtered_anova_df.sort_values(by='month', inplace=True)
        graph_data = filtered_anova_df.groupby('month')['events'].agg(['mean', 'std', 'count', 'sem']).reset_index()
        graph_data.columns = ['month', 'events_mean', 'events_std', 'count', 'sem']
        #graph_data['confidence_interval'] = graph_data.apply(lambda row: st.norm.interval(alpha=0.90, row['count']-1, loc=row['events_mean'], scale=row['sem']))axis=1
        #graph_data['confidence_interval'] = graph_data.apply(lambda row: st.norm.interval(0.90,loc=row['events_mean'],scale=row['sem']),axis=1, if row['events_mean'] != 0 else (0, 0), axis=1)
        graph_data['confidence_interval'] = graph_data.apply(
    lambda row: st.norm.interval(0.90, loc=row['events_mean'], scale=row['sem']) if row['events_mean'] != 0 else (0, 0),
    axis=1
)
        #scale=row['events_std'] / np.sqrt(row['count'])), axis=1)
        graph_data['country'] = temp_country
        graph_data[sub] = temp_crop
        

        # run Tukey test
        tukey = pairwise_tukeyhsd(endog=filtered_anova_df['events'], groups=filtered_anova_df['month'], alpha=0.1)
        if tukey.reject.any():
            print(row['country'], row[sub], ' valid')
            tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
            tukey_df['country'] = row['country']
            tukey_df[sub] = row[sub]
            

            # Get the result summary as a DataFrame
            tukey_summary = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])

            #CLD algarythem 
            cld_data = cld_al(tukey_summary, alpha=0.1)

            #merge CLD with data 
            merged_df = pd.merge(graph_data, cld_data, left_on='month', right_on='groups')
            merged_df = merged_df[['country', sub ,'month', 'events_mean', 'events_std','confidence_interval', 'labels', 'groups']]
            

            #concatonate tables
            tukey_results = (tukey_df.copy() if tukey_results.empty else pd.concat([tukey_results, tukey_df], ignore_index=True))
            graph_data_df = (merged_df.copy() if graph_data_df.empty else pd.concat([graph_data_df, merged_df], ignore_index=True))
           
            
            
        else:
            
            print(row['country'], row[sub] + ' False')


tukey_results.to_csv(sub+' tukey_results.csv', index=False)

print('complete')
        

<font size="5"> Plot graphs with CLD and confidence intervals</font>

In [ ]:
#create plots and save them to pdf
pdffile = PdfPages(sub+' graph.pdf')

#graph_data_df.sort_values(by=sub, inplace=True)
plt.figure()

country_index = 0

for country in graph_data_df['country'].unique():
    country_index += 1
    print('country ', country_index, '/', len(graph_data_df['country'].unique()), ' Started.')
    sub_index = 0

    temp_filterd_graph_df = graph_data_df[graph_data_df['country'] == country]
    

    for subject in temp_filterd_graph_df[sub].unique():     
        sub_index +=1
        print('Subject ', sub_index,'/', len(temp_filterd_graph_df[sub].unique()), 'in country ', country_index,'/',len(graph_data_df['country'].unique()), ' Started.')
        
        filterd_graph_df = temp_filterd_graph_df [temp_filterd_graph_df[sub] == subject]

        
        x_values = filterd_graph_df['month']
        y_values = filterd_graph_df['events_mean']
        std_values = filterd_graph_df['events_std']
        data_label = filterd_graph_df['labels']
        lower_ci = filterd_graph_df['confidence_interval'].apply(lambda x: x[0])
        upper_ci = filterd_graph_df['confidence_interval'].apply(lambda x: x[1])
        confidence_interval = [y_values - lower_ci, upper_ci - y_values]

        # Plotting the bar chart with confidence intervals as error bars
        plt.bar(x_values, y_values, yerr = confidence_interval, capsize=5, label='Events', color='blue')
                
        # Adding data labels
        for x, y, label in zip(x_values, y_values, data_label):
            plt.text(x, y + 0.02, f'{label}', ha='center', va='bottom') 

        # Adding labels and title    
        plt.xlabel('Month')
        plt.ylabel('Events')
        plt.title(country + subject)
        plt.legend()
        pdffile.savefig()
        plt.close()
    
pdffile.close()
print('complete')